### Aufbau der Prolog Programme in Bash:
```
cat <<EOF > filename.ecl
r:- [filename].
% Wissen
EOF

eclipse -f filename.ecl <<EOF
% Anfrage
EOF
```

#### Dokumentation: http://www.eclipseclp.org/doc/bips/fullindex.html

### Beispiel : Tiere
##### Wissen:
* (Alle) Pflanzenfresser sind Säugetiere
* (Alle) Säugetiere sind Tiere
* Jumbo ist ein Pflanzenfresser

In [1]:
# Anfrage: Welche Tiere gibt es alle in unserem System?

# Wir definieren die Relationen und Fakten:
cat <<EOF > saeugetier.ecl
r:- [saeugetier].
saeugetier(X) :- pflanzenfresser(X).
tier(X) :- saeugetier(X).
pflanzenfresser(jumbo).
EOF

# Die Datei mit dem 'Wissen' wird geladen
# und dann können wir anfragen an Prolog stellen:
eclipse -f saeugetier.ecl <<EOF
?-tier(X).
EOF

# Aktuell findet Prolog allerdings nur 1 Lösung :/

ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclipse-clp
GMP library copyright Free Software Foundation, see legal/lgpl.txt
For other libraries see their individual copyright notices
Version 6.1 #164 (x86_64_macosx), Mon Jul 15 23:37 2013
[eclipse 1]: 
X = jumbo
Yes (0.00s cpu)
[eclipse 2]: 
bye


### Beispiel : Berlin
##### Wissen:
* Regierungssitz ist in der Hauptstadt
* Berlin ist Hauptstadt

In [2]:
cat <<EOF > berlin.ecl
r:- [berlin].
regierungssitz(berlin) :- hauptstadt(berlin).
hauptstadt(berlin).
EOF

eclipse -f berlin.ecl <<EOF
?-regierungssitz(berlin).
?-regierungssitz(X).
EOF

ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclipse-clp
GMP library copyright Free Software Foundation, see legal/lgpl.txt
For other libraries see their individual copyright notices
Version 6.1 #164 (x86_64_macosx), Mon Jul 15 23:37 2013
[eclipse 1]: 
Yes (0.00s cpu)
[eclipse 2]: 
X = berlin
Yes (0.00s cpu)
[eclipse 3]: 
bye


### Nutzen von BIP (Built-In-Prozeduren)

##### dynamic pred/1
This directive makes it possible to alter the definition of Pred by adding or removing clauses  
*dynamic => veränderbar*


##### asserta(clause)
Add specified clause at the end of the dynamic procedure to the database  
*assert => behaupten (von einer Aussage)*

##### retract(clause)
Removes the first clause that matches the argument from the database  
*retract => zurücknehmen (von einer Aussage)*

##### !
Cut - succeeds and removes all choice points between cut and parent goal.  
*ist die Aussage bis hierher erfolgreich, dann wird alles andere 'abgeschnitten'*

### Beispiel : FlipFlop

In [10]:
cat <<EOF > einaus.ecl
r:- [einaus].
% Wir definieren Zustand als Dynamic,
% damit dieser während der Laufzeit von Prolog geändert werden kann.
:- dynamic zustand/1.
zustand(ein). % wir setzen initial auf "ein"
einaus(S) :- retract(zustand(ein)),S=aus, asserta(zustand(S)),!.
% ! bedeutet: Wenn der Wechsel auf 'aus' erfolgreich war, wird das Programm beendet.
einaus(S) :- retract(zustand(aus)),S=ein, asserta(zustand(S)).
EOF

eclipse -f einaus.ecl <<EOF
?-einaus(X).
?-einaus(X).
EOF

ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclipse-clp
GMP library copyright Free Software Foundation, see legal/lgpl.txt
For other libraries see their individual copyright notices
Version 6.1 #164 (x86_64_macosx), Mon Jul 15 23:37 2013
[eclipse 1]: 
X = aus
Yes (0.00s cpu)
[eclipse 2]: 
X = ein
Yes (0.00s cpu)
[eclipse 3]: 
bye


### Beispiel: Verwandschaft
##### Wissen / Relationen
* Tom und Maria sind Eltern von Peter
* Lotte ist die Schwester von Peter
* Ralf ist der Bruder von Maria.

##### Frage: Ist Ralf Onkel von Peter?

In [26]:
# tochter(X,Y) :- weiblich(Y), kind(X,Y).
# z.B.: Eine kleine Familie:

cat <<EOF > verwandt.ecl
r:- [verwandt].
maennlich(tom). % Tom ist Maennlich
maennlich(peter). % Peter ist Maennlich
weiblich(maria). % Maria ist weiblich
weiblich(lotte). % Lotte ist weiblich
kind(peter, tom). % Peter ist Kind von Tom.
kind(peter, maria). % Peter ist Kind von Maria
kind(lotte, tom). % Lotte ist kind von Tom
kind(lotte, maria). % Lotte ist kind von Maria

vater(X,Y) :- kind(Y,X), maennlich(X). % Ist X Vater von Y?
mutter(X,Y):- kind(Y,X), weiblich(X). % Ist X Mutter von Y?
sohn(X,Y) :- kind(X,Y), maennlich(X). % Ist X Sohn von Y?
tochter(X,Y) :- kind(X,Y), weiblich(X). % Ist X Tochter von Y?

EOF

RED='\033[0;31m'
NC='\033[0m' # No Color

echo -e "${RED}Ist Peter der Sohn von Tom ?${NC}"
eclipse -f verwandt.ecl <<EOF
?- sohn(peter,tom).
EOF

echo -e "${RED}Ist Tom der Vater von Lotte ?${NC}"
eclipse -f verwandt.ecl <<EOF
?- vater(tom,lotte).
EOF

Ist Peter der Sohn von Tom ?
ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclipse-clp
GMP library copyright Free Software Foundation, see legal/lgpl.txt
For other libraries see their individual copyright notices
Version 6.1 #164 (x86_64_macosx), Mon Jul 15 23:37 2013
[eclipse 1]: 
Yes (0.00s cpu, solution 1, maybe more) ? reading past the file end in tyi(toplevel_input, _276)
Abort
illegal stream specification in set_stream_property(toplevel_input, prompt, '\t')
Ist Tom der Vater von Lotte ?
ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclips

: 2